In [2]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torchvision.transforms as transforms

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

from torch import nn


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\lg_capstone\lg_capstone\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\lg_capstone\lg_capstone\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\lg_capstone\lg_capstone\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "c:\lg_capstone\lg_capstone\Lib\s

In [12]:
torch.__version__

'2.4.0+cu118'

In [13]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)

cuda


## 데이터셋 불러오기

In [7]:
df = pd.read_csv("all_data.csv")

# target 열에 데이터가 있으면 train, 없으면 final 나누기
train = df[df.target.notnull()]
final = df[df.target.isnull()]

C:\Users\jdh25\AppData\Local\Temp\ipykernel_20032\3332094046.py:1: DtypeWarning: Columns (32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("all_data.csv")


In [8]:
#test and split 임포트
from sklearn.model_selection import train_test_split

#데이터셋 분리
train, test = train_test_split(train, test_size=0.2)

In [9]:
#데이터로더 만들기
train_loader = DataLoader(dataset=train,
                          batch_size=128,
                          shuffle=True)

test_loader = DataLoader(dataset=test,
                          batch_size=128,
                          shuffle=True)

## Fully Connected Model

In [10]:
# 28x28x1 => 10
#sequential 순차적으로 블럭을 쌓아서 만든다. 플래튼부터 소프트맥스까지 하나씩 거쳐가는 형식이다.
#보통은 시퀀셜이 아닌 클래스로 모델을 만든다.
mnist_fc_model = nn.Sequential(
    nn.Flatten(), #28x28x1을 784로 펼치는 역할 내가 넣으려는 데이터는 2차원이 아니므로 필요 없을듯
    nn.Linear(in_features=48, out_features=256), #in_features, out_features를 맞추주는 것이 중요
    nn.ReLU(), #요즘은 렐루, 리켈루를 많이 쓴다. 무난하게는 ReLU함수 사용
    nn.Linear(in_features=256, out_features=2), #나의 경우는 아웃피처가 2가 되어야함(정상 불량)
    nn.Softmax() #꼭 해줘야함. 확률값으로 바꿔주는 것
)
#다항함수 하나가 Linear고 2개를 쌓은 모습이다.

In [14]:
# 만든 모델을 device(cuda)로 전달하자
#device는 위에 cuda, cpu 부분 참고
#연산을 GPU의 VRAM으로 모델을 보내줌 데이터는 아직 안보냄 후에 배치로 데이터를 보낼 것이다.
mnist_fc_model.to(device)

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): Linear(in_features=48, out_features=256, bias=True)
  (2): ReLU()
  (3): Linear(in_features=256, out_features=2, bias=True)
  (4): Softmax(dim=None)
)

### 실제 학습이 일어나는 train() 함수

In [15]:
from torch import optim
import time

#로더는 일종의 클래스다. 데이터를 뱉어줌.
def train(model, train_loader):
  # 학습할 epochs수를 정하기
  epochs = 10

  optimizer = optim.Adam(model.parameters(), lr=0.0001)
  #로스를 어떻게 쓸건지 정해주는 것
  criterion = nn.CrossEntropyLoss()
  #시간을 측정하기 위한 코드
  start_time = time.time()

  for epoch in range(epochs):
    # model을 train모드로 변경
    #모델은 훈련상태, 평가상태가 있다. 훈련상태에서는 가중치를 업데이트하고 평가상태에서는 가중치를 업데이트하지 않는다.
    model.train()
    print(f'epochs:{str(epoch+1)} / {str(epochs)}')

    #train_loader를 만들어주는 이유, 중요한 부분
    #samples에 데이터와 정답이 자동으로 뽑혀져 나온다.
    for samples in train_loader:
      x_t, y_t = samples
      # 이 데이터들도 device(cuda)로 보내자
      x_t = x_t.to(device)
      y_t = y_t.to(device)

      pred = model(x_t) #순전파 진행, 예측값을 가져온다.
      # pred랑 실제 값이랑 얼마나 차이나는지 = loss
      loss = criterion(pred, y_t)

      # optimizer를 이용해서 학습(역전파)을 진행
      optimizer.zero_grad() # optimizer를 초기화
      loss.backward() # 역전파 진행
      optimizer.step() # 역전파된 것을 기반으로 가중치 업데이트

    # 하나의 epoch에 대해서 학습이 완료
    # 현재 성능이 어느정도인지 확인하는 코드
    model.eval() # model을 이제 평가하는 모드로 변경
    correct = 0
    for samples in train_loader:
      xx, yy = samples
      xx = xx.to(device)
      yy = yy.to(device) # 5
      pred = model(xx) # [0, 0, 0, 0.1, 0.1, 0, 0, 0.8, 0, 0]
      #언더바는 리턴으로 나오는 값을 받지 않겠다는 의미
      _, predicted = torch.max(pred, 1) # 7
      correct += predicted.eq(yy.data).sum()

    print(f'train_accuracy: {(100. * correct / len(train_loader.dataset)).item()}')

    # gpu랑 싱크를 맞춰줘야하기 때문에 아래 코드를 넣는다
    torch.cuda.synchronize() #GPU가 도는 동안 CPU가 기다려서 time.time()실행 대기를 시켜주는 역할
    end_time = time.time()
    지난시간 = end_time - start_time
    분 = int(지난시간 // 60) # // 몫
    초 = int(지난시간 % 60) # % 나머지
    print(f'현재까지 학습하는데 걸린 시간: {분}분 {초}초')

In [16]:
train(mnist_fc_model, train_loader)

epochs:1 / 10


KeyError: 24094